In [1]:
import itertools
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf, StorageLevel
import sys

In [2]:
conf = SparkConf()
# conf.set("spark.driver.memory", "4g")
# conf.set("spark.executor.memory", "4g")
conf.setMaster('local[8]')
conf.setAppName('Assignment_2')
sc = SparkContext.getOrCreate(conf)

In [23]:
def get_partition_itemset_candidates(bucket, set_size, previous_candidates, total_baskets_count, support):
#     partition_support = int(support * (len(list(bucket)) / total_baskets_count))
    partition_support = 2
    print(partition_support)
    cand_items = {}
    cand_list = []
    
    if set_size > 2:
        if previous_candidates:
            for i in range(len(previous_candidates)):
                for j in range(i+1, len(previous_candidates)):
                    superset = previous_candidates[i].union(previous_candidates[j])
                    if len(superset) == set_size and superset not in cand_items:
                        cand_items[superset] = 0
        
    for basket in bucket:
#         print(basket)
        if set_size > 2:
            for cand in cand_items:
                if cand.issubset(basket):
                    cand_items[cand] += 1
            
        else:
            if set_size == 1:
                sets = basket
                
            else:
                sets = itertools.combinations(basket, set_size)
            
            for item in sets:
#                 print(item)
                if item not in cand_items:
                    cand_items[item] = 0

                cand_items[item] += 1
#     print(cand_items)
    for cand, cand_sup in cand_items.items():
        if cand_sup >= partition_support:
            yield cand
            

In [35]:
def get_original_itemset_counts(basket, support, candidates_list):
#     print(basket)
#     print(candidates_list[:2])
    print(candidates_list)
    if set_size == 1:
        candidates_list = [set([x]) for x in candidates_list]
        print(candidates_list)
    else:
        candidates_list = [set(x) for x in candidates_list]
    for cand in candidates_list:
#         print(cand)
        if cand.issubset(basket):
            yield(tuple(cand), 1)

In [25]:
def get_original_frequent_sets(original_baskets, support, candidate_list):
    original_freq_sets = original_baskets \
        .flatMap(lambda x : get_original_itemset_counts(x, support, candidate_list)) \
        .reduceByKey(lambda a, b: a + b) \
        .filter(lambda x: x[1] >= support)\
        .map(lambda x: (x[0]))\
        .collect()

    return original_freq_sets


In [26]:
def get_candidates_list(candidate_baskets, set_size, previous_candidates, total_baskets_count, support):
    candidate_sets = candidate_baskets.mapPartitions(lambda bucket: get_partition_itemset_candidates(bucket, set_size, previous_candidates, total_baskets_count, support))
    candidate_sets_list = candidate_sets.collect()
    return candidate_sets_list


In [27]:
import time

start_time = time.time()
threshold = 1
case = '1'
support = 4

In [28]:
data_path = 'asnlib/publicdata/'
# data_path = ''

data = sc.textFile(data_path + 'small1.csv').map(lambda x: x.split(',')).map(lambda x: (x[0], x[1]))
header = data.first()
raw_data = data.filter(lambda x: x != header)


if case == '2':
    raw_data = raw_data.map(lambda x: (x[1], x[0]))

baskets = raw_data.groupByKey().map(lambda x: (x[0], set(x[1]))).filter(lambda x: len(x[1]) >= threshold).map(lambda x: x[1])
# print(baskets.collect())
# baskets.persist(StorageLevel.DISK_ONLY)
count_baskets = baskets.count()

In [29]:
count_baskets

19

In [36]:
candidates = []
frequent_itemsets = []
set_size = 1
while True:
    current_candidates = get_candidates_list(
        candidate_baskets=baskets,
        set_size=set_size,
        previous_candidates=candidates[-1] if candidates else [],
        total_baskets_count=count_baskets,
        support=support
    )
    
    print('size_of_set : {}'.format(set_size))
    print('size_of_candidates : {}'.format(len(current_candidates)))
    
    candidates.append(current_candidates)
    
    current_frequent_itemsets = get_original_frequent_sets(
        original_baskets=baskets,
        support=support,
        candidate_list=current_candidates
    )
    if not current_frequent_itemsets:
        break

    frequent_itemsets.append(current_frequent_itemsets)
        
    print('size_of_set : {}'.format(set_size))
    print('size_of_candidates : {}'.format(len(current_candidates)))
    print('size_of_frequent_itemsets : {}'.format(len(current_frequent_itemsets)))
    
    set_size += 1

size_of_set : 1
size_of_candidates : 15
size_of_set : 1
size_of_candidates : 15
size_of_frequent_itemsets : 7
size_of_set : 2
size_of_candidates : 31
size_of_set : 2
size_of_candidates : 31
size_of_frequent_itemsets : 14


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 53.0 failed 1 times, most recent failure: Lost task 1.0 in stage 53.0 (TID 77, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "D:\programming\spark_home\spark-2.3.2-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 253, in main
  File "D:\programming\spark_home\spark-2.3.2-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 248, in process
  File "D:\programming\spark_home\spark-2.3.2-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\serializers.py", line 379, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-23-b13d3e9c065c>", line 12, in get_partition_itemset_candidates
AttributeError: 'tuple' object has no attribute 'union'

	at org.apache.spark.api.python.BasePythonRunner$Re

In [ ]:
len(baskets.take(2)[1])

In [ ]:
baskets.getNumPartitions()

In [ ]:
num_baskets = baskets.count()

In [ ]:
baskets.glom().map(len).collect()

In [ ]:
raw_data.glom().map(len).collect()